In [1]:
import boto3
import os
import time

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

lambda_client = boto3.client('lambda', region_name = 'eu-central-1')

with open("lambda_function.zip", "rb") as f:
    zip_data = f.read()

response = lambda_client.create_function(
    FunctionName='ProcessNewS3Record',
    Runtime='python3.8',
    Role='arn:aws:iam::547303425769:role/MyLambdaS3AndRDSFullAccessRole',
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': zip_data},
    Timeout=10,
    MemorySize=3008,
    Layers=[
        'arn:aws:lambda:eu-central-1:770693421928:layer:Klayers-p38-aws-psycopg2:1',
        'arn:aws:lambda:eu-central-1:770693421928:layer:Klayers-p38-boto3:14'
    ]
)

lambda_function_arn = response['FunctionArn']

time.sleep(15)

lambda_client.add_permission(
    FunctionName='ProcessNewS3Record',
    StatementId='s3-trigger-permission',
    Action='lambda:InvokeFunction',
    Principal='s3.amazonaws.com',
    SourceArn='arn:aws:s3:::daily-bucket-fraud-predicted-manuel',
    SourceAccount='547303425769'
)

s3_client = boto3.client('s3', region_name = 'eu-central-1')

response = s3_client.put_bucket_notification_configuration(
    Bucket='daily-bucket-fraud-predicted-manuel',
    NotificationConfiguration={
        'LambdaFunctionConfigurations': [
            {
                'LambdaFunctionArn': lambda_function_arn,
                'Events': ['s3:ObjectCreated:*'],
            }
        ]
    }
)